<a href="https://colab.research.google.com/github/SeongeunKim-sonja/Keras_example_study_2023/blob/main/Video/Video_Classification_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transformer를 이용한 비디오 분류 - 뒷부분 추가중

*이 문서는 keras 예제를 분석하여 해설을 작성한 것입니다.*

<br>[예제 페이지](https://keras.io/examples/vision/video_transformers/)
<br>[Attention Is All You Need](https://arxiv.org/abs/1706.03762)

<br>
<br>
이 문서에서는 CNN-RNN 모델을 이용한 비디오 분류의 후속으로, Transformer 기반의 모델을 이용하여 비디오를 분류한다. 


이 예제는 TensorFlow 2.5 이상과 TensorFlow Docs를 필요로 한다.


In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

#Data

UCF101 데이터셋 : UCF101 - Action Recognition Data Set
<br>[데이터셋 정보](https://www.crcv.ucf.edu/data/UCF101.php)
<br>101개 동작으로 분류된 13320개의 비디오. 
<br>펀치, 자전거타기 등과 같은 동장으로 분류되어 있으며, 사람의 동작을 인식하는 모델을 생성하는 데에 사용된다. 

In [ ]:
!wget -q https://git.io/JGc31 -O ucf101_top5.tar.gz
!tar xf ucf101_top5.tar.gz

#Setup

In [ ]:
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

#Define hyperparameters

In [ ]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 128

EPOCHS = 5

#Data preparation

- 연산 속도 향상을 위해 224x224크기의 이미지를 128x128 크기로 축소한다.
- 특징 추출을 위해 InceptionV3 네트워크 대신 DenseNet121을 사용한다.  

#DenseNet121

Keras 에서 제공하는 Pre-trained DenseNet model.
<br> 121개의 layer를 가지고 있는 DenseNet을 말한다. 
<br>[DenseNet 참고 블로그](https://gaussian37.github.io/dl-concept-densenet/)


<br>*DenseNet 관련하여 추가하면 좋을 것 같다.*

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)


def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze()
    return cropped


# Following method is modified from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# 다음의 메서드는 위의 tensorflow 에서 제공하는 Inflated 3D Convnet 튜토리얼을 수정한 것이다. 
def load_video(path, max_frames=0):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center(frame)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

#DemseNet121(feature 추출을 위한) 정의의
def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


# Label preprocessing with StringLookup.
# StringLookup 함수를 이용해 라벨링을 진행
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
)
print(label_processor.get_vocabulary())


def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_features` are what we will feed to our sequence model.
    # 'frame_features' 를 시퀀스 모델에 입력할 예정정
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))

        # Pad shorter videos.
        # 제로 프레임을 영상에 덧댄다.
        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
            frames = np.concatenate(frames, padding)

        frames = frames[None, ...]

        # Initialize placeholder to store the features of the current video.
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )

                else:
                    temp_frame_features[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_features.squeeze()

    return frame_features, labels



train_df와 test_df에 prepare_all_videos()를 호출하면 완료되기까지 약 20분이 소요되므로, 시간을 절약하기 위해 미리 처리된 NumPy 배열을 다운로드 한다. 

In [ ]:
!wget -q https://git.io/JZmf4 -O top5_data_prepared.tar.gz
!tar xf top5_data_prepared.tar.gz

In [ ]:
train_data, train_labels = np.load("train_data.npy"), np.load("train_labels.npy")
test_data, test_labels = np.load("test_data.npy"), np.load("test_labels.npy")

print(f"Frame features in train set: {train_data.shape}")

#Building the Transformer-based model